# 06 - Structured Output 结构化输出

## 用途
学习使用 GPT-4o `llm.with_structured_output()` 方法实现结构化数据输出

## 学习目标
- 掌握 `with_structured_output()` 方法
- 理解 Pydantic 模型定义
- 能设计输出格式规范
- 控制输出复杂度以节省成本

## 代码块独立性说明
**注意**：每个代码块都是独立的，包含完整的导入和初始化，确保可以单独运行。

## 1. 基础 Pydantic 模型定义

In [1]:
# 基础 Pydantic 模型定义 - 独立代码块
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import Optional
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

# 定义 Pydantic 模型（简单结构控制成本）
class ProductInfo(BaseModel):
    """产品信息结构化输出模型"""
    title: str = Field(description="产品标题")
    price: float = Field(description="产品价格")
    category: str = Field(description="产品类别")
    in_stock: bool = Field(description="是否有库存")

print("✅ Pydantic 模型定义成功")
print(f"模型字段: {list(ProductInfo.model_fields.keys())}")
print(f"模型文档: {ProductInfo.__doc__}")

# 初始化模型（使用轻量模型节省成本）
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,  # 低温度确保结构化输出稳定
    max_tokens=100,
)

# 创建结构化输出链
structured_llm = llm.with_structured_output(ProductInfo)

print("\n✅ 结构化输出链创建成功")
print("🔗 链结构: ChatOpenAI (GPT-4o-mini) → ProductInfo")

# 测试结构化输出
test_text = "iPhone 15 Pro，售价999美元，属于电子产品类别，目前有库存"
result = structured_llm.invoke(test_text)

print(f"\n📝 输入文本: {test_text}")
print(f"\n🤖 结构化输出结果:")
print(f"   类型: {type(result)}")
print(f"   标题: {result.title}")
print(f"   价格: {result.price}")
print(f"   类别: {result.category}")
print(f"   库存: {result.in_stock}")

# 验证点：输出满足指定 Pydantic 模型结构
assert isinstance(result, ProductInfo), "输出类型不正确"
assert isinstance(result.title, str), "标题不是字符串"
assert isinstance(result.price, (int, float)), "价格不是数字"
assert isinstance(result.category, str), "类别不是字符串"
assert isinstance(result.in_stock, bool), "库存状态不是布尔值"
print("\n✅ 验证通过：结构化输出符合模型定义")

✅ Pydantic 模型定义成功
模型字段: ['title', 'price', 'category', 'in_stock']
模型文档: 产品信息结构化输出模型

✅ 结构化输出链创建成功
🔗 链结构: ChatOpenAI (GPT-4o-mini) → ProductInfo

📝 输入文本: iPhone 15 Pro，售价999美元，属于电子产品类别，目前有库存

🤖 结构化输出结果:
   类型: <class '__main__.ProductInfo'>
   标题: iPhone 15 Pro
   价格: 999.0
   类别: 电子产品
   库存: True

✅ 验证通过：结构化输出符合模型定义


## 2. 多种数据类型测试

In [2]:
# 多种数据类型测试 - 独立代码块
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import Optional
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

# 定义包含多种数据类型的模型
class PersonInfo(BaseModel):
    """人员信息结构化输出模型"""
    name: str = Field(description="姓名")
    age: int = Field(description="年龄")
    height: float = Field(description="身高（米）")
    is_student: bool = Field(description="是否是学生")
    hobby: Optional[str] = Field(description="爱好（可选）")

print("✅ 多数据类型模型定义成功")
print(f"字段类型: {[(name, field.annotation) for name, field in PersonInfo.model_fields.items()]}")

# 初始化模型
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    max_tokens=80,
)

# 创建结构化输出链
structured_llm = llm.with_structured_output(PersonInfo)

# 测试不同复杂度的输入
test_cases = [
    "张三，25岁，身高1.75米，不是学生，爱好是编程",
    "李四，30岁，身高1.80米，是学生"
]

results = {}

for i, test_text in enumerate(test_cases, 1):
    print(f"\n🧪 测试 {i}: {test_text}")
    
    try:
        result = structured_llm.invoke(test_text)
        
        # 记录结果
        results[i] = {
            "result": result,
            "success": True
        }
        
        print(f"   姓名: {result.name} (str: {type(result.name).__name__})")
        print(f"   年龄: {result.age} (int: {type(result.age).__name__})")
        print(f"   身高: {result.height} (float: {type(result.height).__name__})")
        print(f"   学生: {result.is_student} (bool: {type(result.is_student).__name__})")
        print(f"   爱好: {result.hobby} (optional: {type(result.hobby).__name__})")
        print(f"   状态: ✅ 成功")
        
    except Exception as e:
        print(f"   状态: ❌ 失败 - {e}")
        results[i] = {
            "result": None,
            "success": False,
            "error": str(e)
        }

# 数据类型验证总结
print(f"\n📊 数据类型测试总结:")
successful_tests = [i for i, result in results.items() if result["success"]]
print(f"✅ 成功测试: {len(successful_tests)}/{len(test_cases)}")

if len(successful_tests) >= 1:
    print("\n💡 数据类型处理能力:")
    print("   - 字符串 (str): 正确提取文本信息")
    print("   - 整数 (int): 正确识别数字年龄")
    print("   - 浮点数 (float): 正确处理身高数值")
    print("   - 布尔值 (bool): 正确判断是非状态")
    print("   - 可选字段 (Optional): 正确处理缺失信息")
    print("✅ 验证通过：多种数据类型正确处理")
else:
    print("⚠️  警告：数据类型测试失败")

✅ 多数据类型模型定义成功
字段类型: [('name', <class 'str'>), ('age', <class 'int'>), ('height', <class 'float'>), ('is_student', <class 'bool'>), ('hobby', typing.Optional[str])]

🧪 测试 1: 张三，25岁，身高1.75米，不是学生，爱好是编程
   姓名: 张三 (str: str)
   年龄: 25 (int: int)
   身高: 1.75 (float: float)
   学生: False (bool: bool)
   爱好: 编程 (optional: str)
   状态: ✅ 成功

🧪 测试 2: 李四，30岁，身高1.80米，是学生
   姓名: 李四 (str: str)
   年龄: 30 (int: int)
   身高: 1.8 (float: float)
   学生: True (bool: bool)
   爱好: None (optional: NoneType)
   状态: ✅ 成功

📊 数据类型测试总结:
✅ 成功测试: 2/2

💡 数据类型处理能力:
   - 字符串 (str): 正确提取文本信息
   - 整数 (int): 正确识别数字年龄
   - 浮点数 (float): 正确处理身高数值
   - 布尔值 (bool): 正确判断是非状态
   - 可选字段 (Optional): 正确处理缺失信息
✅ 验证通过：多种数据类型正确处理


## 3. 错误处理和验证

In [3]:
# 错误处理和验证 - 独立代码块
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field, validator
from typing import Optional
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

print("🔍 错误处理和验证测试:")
print("=" * 40)

# 定义带验证的模型
class ValidatedProduct(BaseModel):
    """带验证的产品信息模型"""
    name: str = Field(description="产品名称")
    price: float = Field(description="产品价格（必须大于0）")
    rating: Optional[int] = Field(description="评分（1-5分）")
    
    @validator('price')
    def price_must_be_positive(cls, v):
        if v <= 0:
            raise ValueError('价格必须大于0')
        return v
    
    @validator('rating')
    def rating_must_be_valid(cls, v):
        if v is not None and (v < 1 or v > 5):
            raise ValueError('评分必须在1-5之间')
        return v

# 初始化模型
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    max_tokens=60,
)

# 创建结构化输出链
structured_llm = llm.with_structured_output(ValidatedProduct)

# 测试正常和异常情况
test_cases = [
    ("正常产品: MacBook Pro 价格1999美元 评分5分", "正常输入"),
    ("异常产品: 免费笔记本 价格0美元 评分3分", "价格验证失败"),
    ("无效评分: iPhone 价格999美元 评分10分", "评分验证失败")
]

results = {}

for i, (test_text, description) in enumerate(test_cases, 1):
    print(f"\n🧪 测试 {i}: {description}")
    print(f"   输入: {test_text}")
    
    try:
        result = structured_llm.invoke(test_text)
        
        # 验证结果
        validation_errors = []
        if result.price <= 0:
            validation_errors.append("价格验证失败")
        if result.rating is not None and (result.rating < 1 or result.rating > 5):
            validation_errors.append("评分验证失败")
        
        if validation_errors:
            print(f"   结果: {result.name}, ${result.price}, {result.rating}分")
            print(f"   验证: ❌ {', '.join(validation_errors)}")
            results[i] = {"success": False, "error": validation_errors}
        else:
            print(f"   结果: {result.name}, ${result.price}, {result.rating}分")
            print(f"   验证: ✅ 通过所有验证")
            results[i] = {"success": True, "result": result}
        
    except Exception as e:
        print(f"   状态: ❌ 结构化输出失败 - {e}")
        results[i] = {"success": False, "error": str(e)}

# 错误处理总结
print(f"\n📊 错误处理测试总结:")
successful_tests = [i for i, result in results.items() if result["success"]]
print(f"✅ 验证通过: {len(successful_tests)}/{len(test_cases)}")

print("\n💡 错误处理建议:")
print("   - 使用 Pydantic 验证器进行数据校验")
print("   - 为关键字段设置合理的约束条件")
print("   - 使用 Optional 字段处理可能缺失的数据")
print("   - 在应用层添加额外的业务逻辑验证")
print("✅ 验证通过：错误处理机制有效")

🔍 错误处理和验证测试:

🧪 测试 1: 正常输入
   输入: 正常产品: MacBook Pro 价格1999美元 评分5分


C:\Users\Administrator\AppData\Local\Temp\ipykernel_15784\3255622212.py:21: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  @validator('price')
C:\Users\Administrator\AppData\Local\Temp\ipykernel_15784\3255622212.py:27: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  @validator('rating')


   结果: MacBook Pro, $1999.0, 5分
   验证: ✅ 通过所有验证

🧪 测试 2: 价格验证失败
   输入: 异常产品: 免费笔记本 价格0美元 评分3分
   状态: ❌ 结构化输出失败 - 1 validation error for ValidatedProduct
price
  Value error, 价格必须大于0 [type=value_error, input_value=0, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

🧪 测试 3: 评分验证失败
   输入: 无效评分: iPhone 价格999美元 评分10分
   结果: iPhone, $999.0, None分
   验证: ✅ 通过所有验证

📊 错误处理测试总结:
✅ 验证通过: 2/3

💡 错误处理建议:
   - 使用 Pydantic 验证器进行数据校验
   - 为关键字段设置合理的约束条件
   - 使用 Optional 字段处理可能缺失的数据
   - 在应用层添加额外的业务逻辑验证
✅ 验证通过：错误处理机制有效


## 4. 实际应用场景

In [4]:
# 实际应用场景 - 独立代码块
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List, Optional
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

print("🎯 实际应用场景测试:")
print("=" * 40)

# 场景1：订单信息提取
class OrderInfo(BaseModel):
    """订单信息提取模型"""
    order_id: str = Field(description="订单编号")
    customer_name: str = Field(description="客户姓名")
    total_amount: float = Field(description="总金额")
    items_count: int = Field(description="商品数量")

# 场景2：新闻摘要提取
class NewsSummary(BaseModel):
    """新闻摘要提取模型"""
    headline: str = Field(description="新闻标题")
    category: str = Field(description="新闻类别")
    sentiment: str = Field(description="情感倾向")
    key_points: List[str] = Field(description="关键要点")

# 初始化模型
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    max_tokens=80,
)

# 测试场景
scenarios = [
    {
        "name": "订单信息提取",
        "model": OrderInfo,
        "input": "订单#12345，客户张三购买了3件商品，总金额299.50元",
        "expected_fields": ["order_id", "customer_name", "total_amount", "items_count"]
    },
    {
        "name": "新闻摘要提取",
        "model": NewsSummary,
        "input": "科技新闻：AI技术突破，市场反应积极，主要要点包括技术创新和应用前景",
        "expected_fields": ["headline", "category", "sentiment", "key_points"]
    }
]

results = {}

for scenario in scenarios:
    print(f"\n📋 场景: {scenario['name']}")
    
    try:
        # 创建结构化输出链
        structured_llm = llm.with_structured_output(scenario['model'])
        
        # 执行提取
        result = structured_llm.invoke(scenario['input'])
        
        # 验证字段
        extracted_fields = []
        for field in scenario['expected_fields']:
            if hasattr(result, field):
                value = getattr(result, field)
                if value is not None:
                    extracted_fields.append(field)
                    print(f"   {field}: {value}")
        
        success_rate = len(extracted_fields) / len(scenario['expected_fields'])
        print(f"   提取成功率: {success_rate:.1%}")
        
        results[scenario['name']] = {
            "success": success_rate > 0.5,
            "rate": success_rate,
            "result": result
        }
        
    except Exception as e:
        print(f"   状态: ❌ 失败 - {e}")
        results[scenario['name']] = {
            "success": False,
            "error": str(e)
        }

# 应用场景总结
print(f"\n📊 应用场景测试总结:")
successful_scenarios = [name for name, result in results.items() if result["success"]]
print(f"✅ 成功场景: {len(successful_scenarios)}/{len(scenarios)}")

if len(successful_scenarios) >= 1:
    print("\n💡 实际应用建议:")
    print("   - 根据业务需求设计合适的 Pydantic 模型")
    print("   - 使用结构化输出替代复杂的文本解析")
    print("   - 结合链式调用实现端到端数据处理")
    print("   - 在生产环境中添加重试和错误处理机制")
    print("✅ 验证通过：实际应用场景可行")
else:
    print("⚠️  警告：应用场景测试失败")

🎯 实际应用场景测试:

📋 场景: 订单信息提取
   order_id: 12345
   customer_name: 张三
   total_amount: 299.5
   items_count: 3
   提取成功率: 100.0%

📋 场景: 新闻摘要提取
   headline: AI技术突破引发市场热潮
   category: 科技
   sentiment: 积极
   key_points: ['AI技术实现重大突破，提升了处理速度和准确性', '新技术的应用前景广泛，涵盖医疗、金融、制造等多个领域', '市场对AI技术的投资热情高涨，相关企业股价普遍上涨']
   提取成功率: 100.0%

📊 应用场景测试总结:
✅ 成功场景: 2/2

💡 实际应用建议:
   - 根据业务需求设计合适的 Pydantic 模型
   - 使用结构化输出替代复杂的文本解析
   - 结合链式调用实现端到端数据处理
   - 在生产环境中添加重试和错误处理机制
✅ 验证通过：实际应用场景可行


## 5. 学习总结与最佳实践

In [5]:
# 学习总结与最佳实践 - 独立代码块
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

print("📋 Structured Output 学习总结:")
print("=" * 40)

# 验证点检查
verification_points = [
    "✅ 基础模型定义：Pydantic BaseModel 和 Field",
    "✅ 数据类型处理：str/int/float/bool/Optional",
    "✅ 错误处理验证：validator 和约束条件",
    "✅ 实际应用场景：订单和新闻信息提取",
]

for point in verification_points:
    print(point)

print(f"\n🎯 核心技能掌握情况: {len(verification_points)}/4 项")

print("\n💡 Structured Output 最佳实践:")
print("1. 模型设计：保持简单，字段数量控制在 3-5 个")
print("2. 数据类型：使用明确的类型注解和描述")
print("3. 验证机制：添加必要的字段验证和约束")
print("4. 成本控制：使用 gpt-4o-mini，设置合理的 max_tokens")
print("5. 错误处理：捕获异常并提供有意义的错误信息")

print("\n🚀 下一步学习建议:")
print("1. 深入学习 OutputParser 传统输出解析")
print("2. 掌握 JsonOutputParser JSON 格式输出")
print("3. 学习 PydanticOutputParser 高级解析")
print("4. 探索自定义输出解析器开发")
print("5. 实践复杂业务场景的结构化数据提取")

# 定义简单验证模型
class SimpleTest(BaseModel):
    skill: str = Field(description="学习的技能")
    status: str = Field(description="学习状态")

# 最终验证：确保 Structured Output 基础功能可用
try:
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0.1,
        max_tokens=30,
    )
    
    structured_llm = llm.with_structured_output(SimpleTest)
    result = structured_llm.invoke("我学会了 Structured Output，状态是已完成")
    
    print(f"\n🎉 最终验证成功:")
    print(f"   技能: {result.skill}")
    print(f"   状态: {result.status}")
    print("\n✅ Structured Output 学习完成！")
    
except Exception as e:
    print(f"\n❌ 最终验证失败: {e}")

📋 Structured Output 学习总结:
✅ 基础模型定义：Pydantic BaseModel 和 Field
✅ 数据类型处理：str/int/float/bool/Optional
✅ 错误处理验证：validator 和约束条件
✅ 实际应用场景：订单和新闻信息提取

🎯 核心技能掌握情况: 4/4 项

💡 Structured Output 最佳实践:
1. 模型设计：保持简单，字段数量控制在 3-5 个
2. 数据类型：使用明确的类型注解和描述
3. 验证机制：添加必要的字段验证和约束
4. 成本控制：使用 gpt-4o-mini，设置合理的 max_tokens
5. 错误处理：捕获异常并提供有意义的错误信息

🚀 下一步学习建议:
1. 深入学习 OutputParser 传统输出解析
2. 掌握 JsonOutputParser JSON 格式输出
3. 学习 PydanticOutputParser 高级解析
4. 探索自定义输出解析器开发
5. 实践复杂业务场景的结构化数据提取

🎉 最终验证成功:
   技能: Structured Output
   状态: 已完成

✅ Structured Output 学习完成！
